In [1]:
# Setting up dependencies to be utilized later here.
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np



In [10]:
# Converting .csv file into a dataframe for data manipulation
wa_lottery_to_load = Path("Resources/2021_lottery_data.csv")
wa_lottery_pd = pd.read_csv(wa_lottery_to_load)

# displaying .csv just to see what data we have
wa_lottery_pd

,date,weekday,winning_numbers,powerball,powerplay,jackpot
0,1/2/2021,Sat,3-4-11-41-67,5,2,384000000
1,1/6/2021,Wed,1-20-22-60-66,3,3,410000000
2,1/9/2021,Sat,14-26-38-45-46,13,2,470000000
3,1/13/2021,Wed,4-19-23-25-49,14,2,550000000
4,1/16/2021,Sat,14-20-39-65-67,2,3,640000000
...,...,...,...,...,...,...
118,12/20/2021,Mon,2-13-23-34-66,2,4,363000000
119,12/22/2021,Wed,7-16-19-48-68,15,2,378000000
120,12/25/2021,Sat,27-29-45-55-58,2,3,400000000
121,12/27/2021,Mon,36-38-45-62-64,19,2,416000000


In [ ]:


# pie chart
print(wa_lottery_pd['weekday'].value_counts())
